# Conversor programas iPython para Python
Converte um programa escrito em iPython para Python e mantém os comentários no formato do Doxygen, para fazer a documentação do código.

In [1]:
import os # Lendo os arquivos
import re

## Selecionado arquivos
Localizando arquivos python para correção.

In [2]:
filenames = os.listdir("./")
for x in range(len(filenames)-1, -1, -1):
    if filenames[x].find('.py') == -1 or filenames[x].find('.py~') != -1 or filenames[x].find('converter.py') != -1:
        filenames.pop(x)
filenames

['test2.py']

## Iniciando correção

In [3]:
'''Tudo que estiver entre '#ini-tab' e '#end-tab' será acresentado a tabulação no inicio'''
def inserindoTab(data):
    # Localizando primeiro ini-tab
    while True:
        inicio = -1
        for x in xrange(len(data)):
            if data[x].find('#ini-tab') != -1:
                inicio = x
        if inicio == -1:
            break
        for x in xrange(len(data)):
            if data[x].find('#end-tab') != -1:
                fim = x
        for x in xrange(inicio+1, fim):
            data[x] = '\t' + data[x]
        data.pop(fim)
        data.pop(inicio)
    return data

'''As linhas que tiverem o comando '#edes-iPython' será descomentada e a linha seguinte apagada'''
def removerEdes(data):
    for x in range(len(data)-1, -1, -1):
        if data[x].find('#edes-iPython') != -1:
            data[x] = data[x].replace('#edes-iPython ','')
            data.pop(x+1)
    return data

'''As linhas que tiverem o comando '#des-iPython' será descomentada'''
def removerDes(data):
    for x in range(len(data)-1, -1, -1):
        if data[x].find('#des-iPython') != -1:
            data[x] = data[x].replace('#des-iPython ','')
            data[x] = data[x].replace('#des-iPython\t','')
    return data

'''Tudo que entre '#ini-iPython' e '#end-iPython' será apagado'''
def removeriPython(data):
    for x in range(len(data)-1, -1, -1):
        if data[x].find('#ini-iPython') != -1:
            inicio = x
            del data[inicio:fim+1]
        elif data[x].find('#end-iPython') != -1:
            fim = x
    return data

'''Todos os comandos do plt são apagados, para que ele seja repassado ao codigo deve ser usado matplotlib.pyplot'''
def atualizarMatplotlib(data):
    for x in range(len(data)-1, -1, -1):
        if data[x].find('get_ipython()') != -1 or data[x].find('plt') != -1:
            data.pop(x)
            
    for x in range(len(data)-1, -1, -1):
        if data[x].find('import matplotlib.pyplot') != -1:
            data[x] = 'import matplotlib.pyplot as plt'
        elif data[x].find('matplotlib.pyplot') != -1:
            data[x] = data[x].replace('matplotlib.pyplot', 'plt')
    return data

'''Quando se comenta "#self-iPython NOME VARIAVEL" todas as variaveis com esse nome será colocado o self,
isso até o final do arquivo ou até o "#eself-iPython NOME VARIAVEL"'''
def atualizaSelf(data):
    listSelf = []
    remove = []
    for x in range(len(data)):
        for tag in listSelf:
            comment = None
            if data[x].find("#") != -1:
                data[x], comment = data[x].split("#", 1)
                comment = '#' + comment
            if re.search(r"\b" + re.escape(tag) + r"\b", data[x]):
                data[x] = re.sub(r'\b' + tag + r'\b', 'self.' + tag, data[x])
            if comment != None:
                data[x] = data[x] + comment
        
        if data[x].find('#self-iPython') != -1:
            listSelf.append(data[x][data[x].find('#self-iPython')+14:])
            if data[x].find('#self-iPython') != -1:
                data[x] = data[x][:data[x].find('#self-iPython')-1]
            else:
                remove.append(x)
        
        elif data[x].find('#eself-iPython') != -1:
            listSelf.remove(data[x][data[x].find('#eself-iPython')+15:])
            remove.append(x)
    for index in sorted(remove, reverse=True):
        del data[index]
    return data

'''Comentario markdown com o doxygen'''
def removerDoxygen(data):
    for x in range(len(data)-1, -1, -1):
        if data[x].find('# doxygen-iPython') != -1:
            data.pop(x)
            data.pop(x)
    return data

'''Removendo debug'''
def removerDebug(data):
    for x in range(len(data)-1, -1, -1):
        if data[x].find('#debug-iPython') != -1:
            data.pop(x)
    return data

In [4]:
for file in filenames:
    with open(file) as myfile:
        data=myfile.read()
    data = data.replace('    ', '\t') # Alterando espaços por tabulação
    data = data.split('\n')
    
    for x in range(len(data)-1, -1, -1): # Removendo In[
        if data[x].find('# In[') != -1:
            data.pop(x)
            data.pop(x)
    
    # Inserindo alterações aqui
    funcoes = [
        removerDebug,
        removerDes,
        removerEdes,
        removerDoxygen,
        atualizaSelf,
        removeriPython,
        inserindoTab,
        atualizarMatplotlib,
    ]
    for a in funcoes:
        data = a(data)

    for x in range(len(data)-1, -1, -1): # Removendo dois enter
        if data[x].strip() is '' and data[x-1].strip() is '':
            data.pop(x)
    
    for x in xrange(0, len(data)): # Removendo linha começo arquivo
        if data[x].strip() is not '':
            del data[0:x]
            break

    for x in range(len(data)-1, -1, -1): # Removendo linha final arquivo
        if data[x].strip() is '':
            data.pop(x)
        else:
            break
    
    data = '\n'.join(data)
    data = data.replace('# #', '##') # Corrigindo markdown
    
    with open(file, 'w') as outfile: # Salvando alterações
        outfile.write(data)